In [ ]:
#|default_exp lgb_cv

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

# LightGBMCV

> Time series cross validation with LightGBM.

In [ ]:
#|export
import copy
import os
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from typing import Any, Callable, Dict, Iterable, List, Optional, Sequence, Tuple, Union

import lightgbm as lgb
import numpy as np
import pandas as pd

from mlforecast.core import (
    DateFeature,
    Differences,
    Freq,
    LagTransforms,
    Lags,
    TimeSeries,
)
from mlforecast.forecast import MLForecast
from mlforecast.utils import backtest_splits 

In [ ]:
#|hide
from nbdev import show_doc

In [ ]:
#| exporti
def _mape(y_true, y_pred):
    abs_pct_err = abs(y_true - y_pred) / y_true
    return abs_pct_err.groupby(y_true.index.get_level_values(0), observed=True).mean().mean()

def _rmse(y_true, y_pred):
    sq_err = (y_true - y_pred) ** 2
    return sq_err.groupby(y_true.index.get_level_values(0), observed=True).mean().pow(0.5).mean()

_metric2fn = {'mape': _mape, 'rmse': _rmse}

def _update(bst, n):
    for _ in range(n):
        bst.update()

def _predict(ts, bst, valid, h, time_col, dynamic_dfs, predict_fn, **predict_fn_kwargs):
    preds = ts.predict(bst, h, dynamic_dfs, predict_fn, **predict_fn_kwargs).set_index(time_col, append=True)
    return valid.join(preds)

def _update_and_predict(ts, bst, valid, n, h, time_col, dynamic_dfs, predict_fn, **predict_fn_kwargs):
    _update(bst, n)
    return _predict(ts, bst, valid, h, time_col, dynamic_dfs, predict_fn, **predict_fn_kwargs)

In [ ]:
#| exporti
CVResult = Tuple[int, float]

In [ ]:
#| export
class LightGBMCV:
    def __init__(
        self,
        freq: Optional[Freq] = None,
        lags: Optional[Lags] = None,
        lag_transforms: Optional[LagTransforms] = None,
        date_features: Optional[Iterable[DateFeature]] = None,
        differences: Optional[Differences] = None,
        num_threads: int = 1,
    ):
        """Create LightGBM CV object.

        Parameters
        ----------
        freq : str or int, optional (default=None)
            Pandas offset alias, e.g. 'D', 'W-THU' or integer denoting the frequency of the series.
        lags : list of int, optional (default=None)
            Lags of the target to use as features.
        lag_transforms : dict of int to list of functions, optional (default=None)
            Mapping of target lags to their transformations.
        date_features : list of str or callable, optional (default=None)
            Features computed from the dates. Can be pandas date attributes or functions that will take the dates as input.
        differences : list of int, optional (default=None)
            Differences to take of the target before computing the features. These are restored at the forecasting step.
        num_threads : int (default=1)
            Number of threads to use when computing the features.
        """            
        self.num_threads = num_threads
        cpu_count = os.cpu_count()
        if cpu_count is None:
            num_cpus = 1
        else:
            num_cpus = cpu_count
        self.bst_threads = max(num_cpus // num_threads, 1)
        self.ts = TimeSeries(freq, lags, lag_transforms, date_features, differences, self.bst_threads)
        
    def __repr__(self):
        return (
            f'{self.__class__.__name__}('
            f'freq={self.ts.freq}, '
            f'lag_features={list(self.ts.transforms.keys())}, '
            f'date_features={self.ts.date_features}, '
            f'num_threads={self.num_threads}, '
            f'bst_threads={self.bst_threads})'
        )
    
    def setup(
        self,
        data: pd.DataFrame,
        n_windows: int,
        window_size: int,
        id_col: str,
        time_col: str,
        target_col: str,
        params: Optional[Dict[str, Any]] = None,
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
        weights: Optional[Sequence[float]] = None,
        metric: Union[str, Callable] = 'mape',
    ):
        """Initialize internal data structures to iteratively train the boosters. Use this before calling partial_fit.
        
        Parameters
        ----------
        data : pandas DataFrame
            Series data in long format.
        n_windows : int
            Number of windows to evaluate.
        window_size : int
            Number of test periods in each window.
        id_col : str
            Column that identifies each serie. If 'index' then the index is used.
        time_col : str
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str
            Column that contains the target.
        params : dict, optional(default=None)
            Parameters to be passed to the LightGBM Boosters.       
        static_features : list of str, optional (default=None)
            Names of the features that are static and will be repeated when forecasting.
        dropna : bool (default=True)
            Drop rows with missing values produced by the transformations.
        keep_last_n : int, optional (default=None)
            Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it.
        weights : sequence of float, optional (default=None)
            Weights to multiply the metric of each window. If None, all windows have the same weight.
        metric : str or callable, default='mape'
            Metric used to assess the performance of the models and perform early stopping.
            
        Returns
        -------
        self : LightGBMCV
            CV object with internal data structures for partial_fit.
        """
        if weights is None:
            self.weights = np.full(n_windows, 1 / n_windows)        
        elif len(weights) != n_windows:
            raise ValueError('Must specify as many weights as the number of windows')
        else:
            self.weights = np.asarray(weights)
        if callable(metric):
            self.metric_fn = metric
            self.metric_name = 'custom_metric'
        else:
            if metric not in _metric2fn:
                raise ValueError(f'{metric} is not one of the implemented metrics: ({", ".join(_metric2fn.keys())})')
            self.metric_fn = _metric2fn[metric]
            self.metric_name = metric

        if id_col != 'index':
            data = data.set_index(id_col)
        
        if np.issubdtype(data[time_col].dtype.type, np.integer):
            freq = 1
        else:
            freq = self.ts.freq
        self.items = []
        self.window_size = window_size
        self.time_col = time_col
        self.target_col = target_col
        params = {} if params is None else params
        for _, train, valid in backtest_splits(data, n_windows, window_size, freq, time_col):
            ts = copy.deepcopy(self.ts)
            prep = ts.fit_transform(train, 'index', time_col, target_col, static_features, dropna, keep_last_n)
            ds = lgb.Dataset(prep.drop(columns=[time_col, target_col]), prep[target_col]).construct()
            bst = lgb.Booster({**params, 'num_threads': self.bst_threads}, ds)
            bst.predict = partial(bst.predict, num_threads=self.bst_threads)
            valid = valid.set_index(time_col, append=True)
            self.items.append((ts, bst, valid))
        return self

    def _single_threaded_partial_fit(
        self,
        metric_values,
        num_iterations,
        dynamic_dfs,
        predict_fn,
        **predict_fn_kwargs,
    ):  
        for j, (ts, bst, valid) in enumerate(self.items):                        
            preds = _update_and_predict(
                ts,
                bst,
                valid,
                num_iterations,
                self.window_size,
                self.time_col,
                dynamic_dfs,
                predict_fn,
                **predict_fn_kwargs
            )
            metric_values[j] = self.metric_fn(preds[self.target_col], preds['Booster'])

    def _multithreaded_partial_fit(
        self,
        metric_values,
        num_iterations,
        dynamic_dfs,
        predict_fn,
        **predict_fn_kwargs,
    ):                           
        with ThreadPoolExecutor(self.num_threads) as executor:
            futures = []
            for ts, bst, valid in self.items:
                _update(bst, num_iterations)
                future = executor.submit(
                    _predict,
                    ts,
                    bst,
                    valid,
                    self.window_size,
                    self.time_col,
                    dynamic_dfs,
                    predict_fn,
                    **predict_fn_kwargs
                )
                futures.append(future)
            cv_preds = [f.result() for f in futures]
        metric_values[:] = [self.metric_fn(preds[self.target_col], preds['Booster']) for preds in cv_preds]
        
    def partial_fit(
        self,
        num_iterations: int,
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,
        predict_fn: Optional[Callable] = None,
        **predict_fn_kwargs,
    ) -> float:
        """Train the boosters for some iterations.
        
        Parameters
        ----------
        num_iterations : int
            Number of boosting iterations to run
        dynamic_dfs : list of pandas DataFrame, optional (default=None)
            Future values of the dynamic features, e.g. prices.
        predict_fn : callable, optional (default=None)
            Custom function to compute predictions.
            This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,
            and should return an array with the predictions, where:
                model : regressor
                    Fitted model.
                new_x : pandas DataFrame
                    Current values of the features.
                dynamic_dfs : list of pandas DataFrame
                    Future values of the dynamic features
                features_order : list of str
                    Column names in the order in which they were used to train the model.
                **kwargs
                    Other keyword arguments passed to `MLForecast.predict`.
        **predict_fn_kwargs
            Additional arguments passed to predict_fn
                    
        Returns
        -------
        metric_value : float
            Weighted metric after training for num_iterations.
        """
        metric_values = np.empty(len(self.items))
        if self.num_threads == 1:
            self._single_threaded_partial_fit(metric_values, num_iterations, dynamic_dfs, predict_fn, **predict_fn_kwargs)
        else:
            self._multithreaded_partial_fit(metric_values, num_iterations, dynamic_dfs, predict_fn, **predict_fn_kwargs)
        return metric_values @ self.weights
    
    def _should_stop(self, hist, early_stopping_evals, early_stopping_pct) -> bool:
        if len(hist) < early_stopping_evals + 1:
            return False
        improvement_pct = 1 - hist[-1][1] / hist[-(early_stopping_evals + 1)][1]
        return improvement_pct < early_stopping_pct

    def _best_iter(self, hist, early_stopping_evals) -> int:
        best_iter, best_score = hist[-1]
        for r, m in hist[-(early_stopping_evals + 1):-1]:
            if m < best_score:
                best_score = m
                best_iter = r
        return best_iter
   
    def fit(
        self,
        data: pd.DataFrame,
        n_windows: int,
        window_size: int,
        id_col: str,
        time_col: str,
        target_col: str,
        num_iterations: int = 100,
        params: Optional[Dict[str, Any]] = None,
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,
        eval_every: int = 10,
        weights: Optional[Sequence[float]] = None,
        metric: Union[str, Callable] = 'mape',
        verbose_eval: bool = True,
        early_stopping_evals: int = 2,
        early_stopping_pct: float = 0.01,
        compute_cv_preds: bool = False,
        fit_on_all: bool = False,
        predict_fn: Optional[Callable] = None,
        **predict_fn_kwargs,
    ) -> List[CVResult]:
        """Train boosters simultaneously and assess their performance on the complete forecasting window.
        
        Parameters
        ----------
        data : pandas DataFrame
            Series data in long format.
        n_windows : int
            Number of windows to evaluate.
        window_size : int
            Number of test periods in each window.    
        id_col : str
            Column that identifies each serie. If 'index' then the index is used.
        time_col : str
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str
            Column that contains the target.
        num_iterations : int (default=100)
            Maximum number of boosting iterations to run.
        params : dict, optional(default=None)
            Parameters to be passed to the LightGBM Boosters.            
        static_features : list of str, optional (default=None)
            Names of the features that are static and will be repeated when forecasting.
        dropna : bool (default=True)
            Drop rows with missing values produced by the transformations.
        keep_last_n : int, optional (default=None)
            Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it.
        dynamic_dfs : list of pandas DataFrame, optional (default=None)
            Future values of the dynamic features, e.g. prices.
        eval_every : int (default=10)
            Number of boosting iterations to train before evaluating on the whole forecast window.
        weights : sequence of float, optional (default=None)
            Weights to multiply the metric of each window. If None, all windows have the same weight.
        metric : str or callable, default='mape'
            Metric used to assess the performance of the models and perform early stopping.
        verbose_eval : bool
            Print the metrics of each evaluation.
        early_stopping_evals : int (default=2)
            Maximum number of evaluations to run without improvement.
        early_stopping_pct : float (default=0.01)
            Minimum percentage improvement in metric value in `early_stopping_evals` evaluations.
        compute_cv_preds : bool (default=True)
            Compute predictions for each window after finding the best iteration.        
        fit_on_all : bool (default=True)
            Return model fitted on full dataset.
        predict_fn : callable, optional (default=None)
            Custom function to compute predictions.
            This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,
            and should return an array with the predictions, where:
                model : regressor
                    Fitted model.
                new_x : pandas DataFrame
                    Current values of the features.
                dynamic_dfs : list of pandas DataFrame
                    Future values of the dynamic features
                features_order : list of str
                    Column names in the order in which they were used to train the model.
                **kwargs
                    Other keyword arguments passed to `MLForecast.predict`.
        **predict_fn_kwargs
            Additional arguments passed to predict_fn                    

        Returns
        -------
        cv_result : list of tuple.
            List of (boosting rounds, metric value) tuples.
        """
        self.setup(
            data=data,
            n_windows=n_windows,
            window_size=window_size,
            params=params,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            static_features=static_features,
            dropna=dropna,
            keep_last_n=keep_last_n,
            weights=weights,
            metric=metric,
        )
        hist = []
        for i in range(0, num_iterations, eval_every):
            metric_value = self.partial_fit(eval_every, dynamic_dfs, predict_fn, **predict_fn_kwargs)
            rounds = eval_every + i
            hist.append((rounds, metric_value))
            if verbose_eval:
                print(f'[{rounds:,d}] {self.metric_name}: {metric_value:,f}')                
            if self._should_stop(hist, early_stopping_evals, early_stopping_pct):
                print(f"Early stopping at round {rounds:,}")
                break
        rounds = self._best_iter(hist, early_stopping_evals)
        print(f'Using best iteration: {rounds:,}')
        hist = hist[:rounds // eval_every]
        for _, bst, _ in self.items:
            bst.best_iteration = rounds

        self.cv_models_ = [item[1] for item in self.items]
        if compute_cv_preds:
            with ThreadPoolExecutor(self.num_threads) as executor:
                futures = []            
                for ts, bst, valid in self.items:
                    future = executor.submit(
                        _predict,
                        ts,
                        bst,
                        valid,
                        window_size,
                        time_col,
                        dynamic_dfs,
                        predict_fn,
                        **predict_fn_kwargs
                    )
                    futures.append(future)            
                self.cv_preds_ = pd.concat([f.result().assign(window=i) for i, f in enumerate(futures)])

        if fit_on_all:
            params = params if params is not None else {}            
            self.fcst = MLForecast([lgb.LGBMRegressor(**{**params, 'n_estimators': rounds})])
            self.fcst.ts = self.ts
            self.fcst.fit(
                data,
                id_col,
                time_col,
                target_col,
                static_features,
                dropna,
                keep_last_n,
            )
        else:
            self.ts._fit(data, id_col, time_col, target_col, static_features, keep_last_n)
        return hist

    def predict(
        self,
        horizon: int,
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,
        predict_fn: Optional[Callable] = None,
        **predict_fn_kwargs,
    ) -> pd.DataFrame:
        """Compute predictions using the model trained on all data.
        
        Parameters
        ----------
        horizon : int
            Number of periods to predict.
        dynamic_dfs : list of pandas DataFrame, optional (default=None)
            Future values of the dynamic features, e.g. prices.
        predict_fn : callable, optional (default=None)
            Custom function to compute predictions.
            This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,
            and should return an array with the predictions, where:
                model : regressor
                    Fitted model.
                new_x : pandas DataFrame
                    Current values of the features.
                dynamic_dfs : list of pandas DataFrame
                    Future values of the dynamic features
                features_order : list of str
                    Column names in the order in which they were used to train the model.
                **kwargs
                    Other keyword arguments passed to `MLForecast.predict`.
        **predict_fn_kwargs
            Additional arguments passed to predict_fn
                    
        Returns
        -------
        result : pandas DataFrame
            Predictions for each serie and timestep.
        """        
        if not hasattr(self, 'fcst'):
            raise ValueError('Must call fit with fit_on_all=True before. You can also call cv_predict instead.')
        return self.fcst.predict(horizon, dynamic_dfs, predict_fn, **predict_fn_kwargs)
    
    def cv_predict(
        self,
        horizon: int,
        dynamic_dfs: Optional[List[pd.DataFrame]] = None,
        predict_fn: Optional[Callable] = None,
        **predict_fn_kwargs,
    ) -> pd.DataFrame:
        """Compute predictions with each of the trained boosters.
        
        Parameters
        ----------
        horizon : int
            Number of periods to predict.
        dynamic_dfs : list of pandas DataFrame, optional (default=None)
            Future values of the dynamic features, e.g. prices.
        predict_fn : callable, optional (default=None)
            Custom function to compute predictions.
            This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,
            and should return an array with the predictions, where:
                model : regressor
                    Fitted model.
                new_x : pandas DataFrame
                    Current values of the features.
                dynamic_dfs : list of pandas DataFrame
                    Future values of the dynamic features
                features_order : list of str
                    Column names in the order in which they were used to train the model.
                **kwargs
                    Other keyword arguments passed to `MLForecast.predict`.
        **predict_fn_kwargs
            Additional arguments passed to predict_fn
                    
        Returns
        -------
        result : pandas DataFrame
            Predictions for each serie and timestep, with one column per window.
        """
        return self.ts.predict(
            self.cv_models_,
            horizon,
            dynamic_dfs,
            predict_fn,
            **predict_fn_kwargs
        )

In [ ]:
show_doc(LightGBMCV)

---

### LightGBMCV

>      LightGBMCV (freq:Union[int,str,NoneType]=None,
>                  lags:Optional[Iterable[int]]=None, lag_transforms:Optional[Di
>                  ct[int,List[Union[Callable,Tuple[Callable,Any]]]]]=None,
>                  date_features:Optional[Iterable[Union[str,Callable]]]=None,
>                  differences:Optional[Iterable[int]]=None, num_threads:int=1)

Create LightGBM CV object.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| freq | typing.Union[int, str, NoneType] | None | Pandas offset alias, e.g. 'D', 'W-THU' or integer denoting the frequency of the series. |
| lags | typing.Optional[typing.Iterable[int]] | None | Lags of the target to use as features. |
| lag_transforms | typing.Optional[typing.Dict[int, typing.List[typing.Union[typing.Callable, typing.Tuple[typing.Callable, typing.Any]]]]] | None | Mapping of target lags to their transformations. |
| date_features | typing.Optional[typing.Iterable[typing.Union[str, typing.Callable]]] | None | Features computed from the dates. Can be pandas date attributes or functions that will take the dates as input. |
| differences | typing.Optional[typing.Iterable[int]] | None | Differences to take of the target before computing the features. These are restored at the forecasting step. |
| num_threads | int | 1 | Number of threads to use when computing the features. |

## Example
This shows an example with just 4 series of the M4 dataset. If you want to run it yourself on all of them, you can refer to [this notebook](https://www.kaggle.com/code/lemuz90/m4-competition-cv).

In [ ]:
import random

from datasetsforecast.m4 import M4, M4Info
from fastcore.test import test_eq, test_fail
from nbdev import show_doc
from window_ops.ewm import ewm_mean
from window_ops.rolling import rolling_mean, seasonal_rolling_mean

In [ ]:
group = 'Hourly'
await M4.async_download('data', group=group)
df, *_ = M4.load(directory='data', group=group)
df['ds'] = df['ds'].astype('int')
ids = df['unique_id'].unique()
random.seed(0)
sample_ids = random.choices(ids, k=4)
sample_df = df[df['unique_id'].isin(sample_ids)]
sample_df

,unique_id,ds,y
86796,H196,1,11.8
86797,H196,2,11.4
86798,H196,3,11.1
86799,H196,4,10.8
86800,H196,5,10.6
...,...,...,...
325235,H413,1004,99.0
325236,H413,1005,88.0
325237,H413,1006,47.0
325238,H413,1007,41.0


In [ ]:
info = M4Info[group]
horizon = info.horizon
valid = sample_df.groupby('unique_id').tail(horizon)
train = sample_df.drop(valid.index)
train.shape, valid.shape

((3840, 3), (192, 3))

What LightGBMCV does is emulate [LightGBM's cv function](https://lightgbm.readthedocs.io/en/v3.3.2/pythonapi/lightgbm.cv.html#lightgbm.cv) where several Boosters are trained simultaneously on different partitions of the data, that is, one boosting iteration is performed on all of them at a time. This allows to have an estimate of the error by iteration, so if we combine this with early stopping we can find the best iteration to train a final model using all the data or even use these individual models' predictions to compute an ensemble.

In order to have a good estimate of the forecasting performance of our model we compute predictions for the whole test period and compute a metric on that. Since this step can slow down training, there's an `eval_every` parameter that can be used to control this, that is, if `eval_every=10` (the default) every 10 boosting iterations we're going to compute forecasts for the complete window and report the error.

We also have early stopping parameters:

* `early_stopping_evals`: how many evaluations of the full window should we go without improving to stop training?
* `early_stopping_pct`: what's the minimum percentage improvement we want in these `early_stopping_evals` in order to keep training?

This makes the LightGBMCV class a good tool to quickly test different configurations of the model. Consider the following example, where we're going to try to find out which features can improve the performance of our model. We start just using lags.

In [ ]:
static_fit_config = dict(
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    n_windows=2,
    window_size=horizon,
    params={'verbose': -1},
    fit_on_all=True,
)
cv = LightGBMCV(
    freq=1,
    lags=[24 * (i+1) for i in range(7)],  # one week of lags
)

In [ ]:
show_doc(LightGBMCV.fit)

---

### LightGBMCV.fit

>      LightGBMCV.fit (data:pandas.core.frame.DataFrame, n_windows:int,
>                      window_size:int, id_col:str, time_col:str,
>                      target_col:str, num_iterations:int=100,
>                      params:Optional[Dict[str,Any]]=None,
>                      static_features:Optional[List[str]]=None,
>                      dropna:bool=True, keep_last_n:Optional[int]=None, dynamic
>                      _dfs:Optional[List[pandas.core.frame.DataFrame]]=None,
>                      eval_every:int=10,
>                      weights:Optional[Sequence[float]]=None,
>                      metric:Union[str,Callable]='mape',
>                      verbose_eval:bool=True, early_stopping_evals:int=2,
>                      early_stopping_pct:float=0.01,
>                      compute_cv_preds:bool=False, fit_on_all:bool=False,
>                      predict_fn:Optional[Callable]=None, **predict_fn_kwargs)

Train boosters simultaneously and assess their performance on the complete forecasting window.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| data | DataFrame |  | Series data in long format. |
| n_windows | int |  | Number of windows to evaluate. |
| window_size | int |  | Number of test periods in each window.     |
| id_col | str |  | Column that identifies each serie. If 'index' then the index is used. |
| time_col | str |  | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str |  | Column that contains the target. |
| num_iterations | int | 100 | Maximum number of boosting iterations to run. |
| params | typing.Optional[typing.Dict[str, typing.Any]] | None | Parameters to be passed to the LightGBM Boosters.             |
| static_features | typing.Optional[typing.List[str]] | None | Names of the features that are static and will be repeated when forecasting. |
| dropna | bool | True | Drop rows with missing values produced by the transformations. |
| keep_last_n | typing.Optional[int] | None | Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it. |
| dynamic_dfs | typing.Optional[typing.List[pandas.core.frame.DataFrame]] | None | Future values of the dynamic features, e.g. prices. |
| eval_every | int | 10 | Number of boosting iterations to train before evaluating on the whole forecast window. |
| weights | typing.Optional[typing.Sequence[float]] | None | Weights to multiply the metric of each window. If None, all windows have the same weight. |
| metric | typing.Union[str, typing.Callable] | mape | Metric used to assess the performance of the models and perform early stopping. |
| verbose_eval | bool | True | Print the metrics of each evaluation. |
| early_stopping_evals | int | 2 | Maximum number of evaluations to run without improvement. |
| early_stopping_pct | float | 0.01 | Minimum percentage improvement in metric value in `early_stopping_evals` evaluations. |
| compute_cv_preds | bool | False | Compute predictions for each window after finding the best iteration.         |
| fit_on_all | bool | False | Return model fitted on full dataset. |
| predict_fn | typing.Optional[typing.Callable] | None | Custom function to compute predictions.<br>This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,<br>and should return an array with the predictions, where:<br>    model : regressor<br>        Fitted model.<br>    new_x : pandas DataFrame<br>        Current values of the features.<br>    dynamic_dfs : list of pandas DataFrame<br>        Future values of the dynamic features<br>    features_order : list of str<br>        Column names in the order in which they were used to train the model.<br>    **kwargs<br>        Other keyword arguments passed to `MLForecast.predict`. |
| predict_fn_kwargs |  |  |  |
| **Returns** | **typing.List[typing.Tuple[int, float]]** |  | **List of (boosting rounds, metric value) tuples.** |

In [ ]:
hist = cv.fit(train, **static_fit_config)

[LightGBM] [Info] Start training from score 51.745632
[10] mape: 0.590690
[20] mape: 0.251093
[30] mape: 0.143643
[40] mape: 0.109723
[50] mape: 0.102099
[60] mape: 0.099448
[70] mape: 0.098349
[80] mape: 0.098006
[90] mape: 0.098718
Early stopping at round 90
Using best iteration: 80


By setting `fit_on_all=True` a final model gets trained with all of the data and the best iteration found, then we can call `predict` to get the predictions from this model.

In [ ]:
show_doc(LightGBMCV.predict)

---

### LightGBMCV.predict

>      LightGBMCV.predict (horizon:int,
>                          dynamic_dfs:Optional[List[pandas.core.frame.DataFrame
>                          ]]=None, predict_fn:Optional[Callable]=None,
>                          **predict_fn_kwargs)

Compute predictions using the model trained on all data.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| horizon | int |  | Number of periods to predict. |
| dynamic_dfs | typing.Optional[typing.List[pandas.core.frame.DataFrame]] | None | Future values of the dynamic features, e.g. prices. |
| predict_fn | typing.Optional[typing.Callable] | None | Custom function to compute predictions.<br>This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,<br>and should return an array with the predictions, where:<br>    model : regressor<br>        Fitted model.<br>    new_x : pandas DataFrame<br>        Current values of the features.<br>    dynamic_dfs : list of pandas DataFrame<br>        Future values of the dynamic features<br>    features_order : list of str<br>        Column names in the order in which they were used to train the model.<br>    **kwargs<br>        Other keyword arguments passed to `MLForecast.predict`. |
| predict_fn_kwargs |  |  |  |
| **Returns** | **DataFrame** |  | **Predictions for each serie and timestep.** |

In [ ]:
preds = cv.predict(horizon)
preds

,ds,LGBMRegressor
unique_id,,
H196,961,15.644404
H196,962,15.571694
H196,963,15.044647
H196,964,14.931199
H196,965,14.547140
...,...,...
H413,1004,69.663252
H413,1005,62.815338
H413,1006,62.073884


We can evaluate these predictions on the validation set.

In [ ]:
def evaluate_on_valid(preds):
    merged = preds.merge(valid, on=['unique_id', 'ds'])
    merged['abs_err'] = abs(merged['LGBMRegressor'] - merged['y']) / merged['y']
    return merged.groupby('unique_id')['abs_err'].mean().mean()

evaluate_on_valid(preds)

0.11214839487071401

The individual models we trained are also saved, so there's also `cv_predict` that uses these models instead.

In [ ]:
show_doc(LightGBMCV.cv_predict)

---

### LightGBMCV.cv_predict

>      LightGBMCV.cv_predict (horizon:int,
>                             dynamic_dfs:Optional[List[pandas.core.frame.DataFr
>                             ame]]=None, predict_fn:Optional[Callable]=None,
>                             **predict_fn_kwargs)

Compute predictions with each of the trained boosters.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| horizon | int |  | Number of periods to predict. |
| dynamic_dfs | typing.Optional[typing.List[pandas.core.frame.DataFrame]] | None | Future values of the dynamic features, e.g. prices. |
| predict_fn | typing.Optional[typing.Callable] | None | Custom function to compute predictions.<br>This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,<br>and should return an array with the predictions, where:<br>    model : regressor<br>        Fitted model.<br>    new_x : pandas DataFrame<br>        Current values of the features.<br>    dynamic_dfs : list of pandas DataFrame<br>        Future values of the dynamic features<br>    features_order : list of str<br>        Column names in the order in which they were used to train the model.<br>    **kwargs<br>        Other keyword arguments passed to `MLForecast.predict`. |
| predict_fn_kwargs |  |  |  |
| **Returns** | **DataFrame** |  | **Predictions for each serie and timestep, with one column per window.** |

In [ ]:
cv_preds = cv.cv_predict(horizon)
cv_preds

,ds,Booster,Booster2
unique_id,,,
H196,961,15.670252,15.848888
H196,962,15.522924,15.697399
H196,963,14.985832,15.166213
H196,964,14.985832,14.723238
H196,965,14.562152,14.451092
...,...,...,...
H413,1004,70.695242,65.917620
H413,1005,66.216580,62.615788
H413,1006,63.896573,67.848598


We can average these predictions and evaluate them.

In [ ]:
cv_preds['LGBMRegressor'] = cv_preds.drop(columns='ds').mean(1)
evaluate_on_valid(cv_preds)

0.11036194712311806

In [ ]:
assert evaluate_on_valid(cv_preds) < evaluate_on_valid(preds)

In this example this achieves a better score!

Now, since these series are hourly, maybe we can try to remove the daily seasonality by taking the 168th (24 * 7) difference, that is, substract the value at the same hour from one week ago, thus our target will be $z_t = y_{t} - y_{t-168}$. The features will be computed from this target and when we predict they will be automatically re-applied.

In [ ]:
cv2 = LightGBMCV(
    freq=1,
    differences=[24 * 7],
    lags=[24 * (i+1) for i in range(7)],
)
hist2 = cv2.fit(train, **static_fit_config)

[LightGBM] [Info] Start training from score 0.519010
[10] mape: 0.089024
[20] mape: 0.090683
[30] mape: 0.092316
Early stopping at round 30
Using best iteration: 10


In [ ]:
assert hist2[-1][1] < hist[-1][1]

Nice! We achieve a better score in less iterations. Let's see if this improvement translates to the validation set as well.

In [ ]:
preds2 = cv2.predict(horizon)
evaluate_on_valid(preds2)

0.0912676397267144

In [ ]:
assert evaluate_on_valid(preds2) < evaluate_on_valid(preds)

Great! Maybe we can try some lag transforms now. We'll try the seasonal rolling mean that averages the values "every season", that is, if we set `season_length=24` and `window_size=7` then we'll average the value at the same hour for every day of the week.

In [ ]:
cv3 = LightGBMCV(
    freq=1,
    differences=[24 * 7],
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        48: [(seasonal_rolling_mean, 24, 7)],
    },
)
hist3 = cv3.fit(train, **static_fit_config)

[LightGBM] [Info] Start training from score 0.273641
[10] mape: 0.086724
[20] mape: 0.088466
[30] mape: 0.090536
Early stopping at round 30
Using best iteration: 10


Seems like this is helping as well!

In [ ]:
assert hist3[-1][1] < hist2[-1][1]

Does this reflect on the validation set?

In [ ]:
preds3 = cv3.predict(horizon)
evaluate_on_valid(preds3)

0.0907707496184505

Nice! mlforecast also supports date features, but in this case our time column is made from integers so there aren't many possibilites here. As you can see this allows you to iterate faster and get better estimates of the forecasting performance you can expect from your model.

In [ ]:
#| hide
test_eq(cv._best_iter([(0, 1), (1, 0.5)], 1), 1)
test_eq(cv._best_iter([(0, 1), (1, 0.5), (2, 0.6)], 1), 1)
test_eq(cv._best_iter([(0, 1), (1, 0.5), (2, 0.6), (3, 0.4)], 2), 3)

If you're doing hyperparameter tuning it's useful to be able to run a couple of iterations, assess the performance, and determine if this particular configuration isn't promising and should be discarded. For example, [optuna](https://optuna.org/) has [pruners](https://optuna.readthedocs.io/en/stable/reference/pruners.html) that you can call with your current score and it decides if the trial should be discarded. We'll now show how to do that.

Since the CV requires a bit of setup, like the LightGBM datasets and the internal features, we have this `setup` method.

In [ ]:
show_doc(LightGBMCV.setup)

---

### LightGBMCV.setup

>      LightGBMCV.setup (data:pandas.core.frame.DataFrame, n_windows:int,
>                        window_size:int, id_col:str, time_col:str,
>                        target_col:str, params:Optional[Dict[str,Any]]=None,
>                        static_features:Optional[List[str]]=None,
>                        dropna:bool=True, keep_last_n:Optional[int]=None,
>                        weights:Optional[Sequence[float]]=None,
>                        metric:Union[str,Callable]='mape')

Initialize internal data structures to iteratively train the boosters. Use this before calling partial_fit.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| data | DataFrame |  | Series data in long format. |
| n_windows | int |  | Number of windows to evaluate. |
| window_size | int |  | Number of test periods in each window. |
| id_col | str |  | Column that identifies each serie. If 'index' then the index is used. |
| time_col | str |  | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str |  | Column that contains the target. |
| params | typing.Optional[typing.Dict[str, typing.Any]] | None | Parameters to be passed to the LightGBM Boosters.        |
| static_features | typing.Optional[typing.List[str]] | None | Names of the features that are static and will be repeated when forecasting. |
| dropna | bool | True | Drop rows with missing values produced by the transformations. |
| keep_last_n | typing.Optional[int] | None | Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it. |
| weights | typing.Optional[typing.Sequence[float]] | None | Weights to multiply the metric of each window. If None, all windows have the same weight. |
| metric | typing.Union[str, typing.Callable] | mape | Metric used to assess the performance of the models and perform early stopping. |
| **Returns** | **LightGBMCV** |  | **CV object with internal data structures for partial_fit.** |

In [ ]:
cv4 = LightGBMCV(
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
)
cv4.setup(
    train,
    n_windows=2,
    window_size=horizon,
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    params={'verbose': -1},
)

LightGBMCV(freq=1, lag_features=['lag-24', 'lag-48', 'lag-72', 'lag-96', 'lag-120', 'lag-144', 'lag-168'], date_features=[], num_threads=1, bst_threads=8)

Once we have this we can call `partial_fit` to only train for some iterations and return the score of the forecast window.

In [ ]:
show_doc(LightGBMCV.partial_fit)

---

### LightGBMCV.partial_fit

>      LightGBMCV.partial_fit (num_iterations:int,
>                              dynamic_dfs:Optional[List[pandas.core.frame.DataF
>                              rame]]=None, predict_fn:Optional[Callable]=None,
>                              **predict_fn_kwargs)

Train the boosters for some iterations.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| num_iterations | int |  | Number of boosting iterations to run |
| dynamic_dfs | typing.Optional[typing.List[pandas.core.frame.DataFrame]] | None | Future values of the dynamic features, e.g. prices. |
| predict_fn | typing.Optional[typing.Callable] | None | Custom function to compute predictions.<br>This function will recieve: model, new_x, dynamic_dfs, features_order and kwargs,<br>and should return an array with the predictions, where:<br>    model : regressor<br>        Fitted model.<br>    new_x : pandas DataFrame<br>        Current values of the features.<br>    dynamic_dfs : list of pandas DataFrame<br>        Future values of the dynamic features<br>    features_order : list of str<br>        Column names in the order in which they were used to train the model.<br>    **kwargs<br>        Other keyword arguments passed to `MLForecast.predict`. |
| predict_fn_kwargs |  |  |  |
| **Returns** | **float** |  | **Weighted metric after training for num_iterations.** |

In [ ]:
score = cv4.partial_fit(10)
score

[LightGBM] [Info] Start training from score 51.745632


0.5906900462828166

This is equal to the first evaluation from our first example.

In [ ]:
assert hist[0][1] == score

We can now use this score to decide if this configuration is promising. If we want to we can train some more iterations.

In [ ]:
score2 = cv4.partial_fit(20)

This is now equal to our third metric from the first example, since this time we trained for 20 iterations.

In [ ]:
assert hist[2][1] == score2

In [ ]:
#| hide
cv5 = LightGBMCV(
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
)
[(_, score3)] = cv5.fit(train, **{**static_fit_config, 'fit_on_all': False}, num_iterations=10, verbose_eval=False)
assert score3 == score
test_fail(lambda: cv5.predict(1), contains='Must call fit with fit_on_all=True')

[LightGBM] [Info] Start training from score 51.745632
Using best iteration: 10
